In [1]:
#importing the libraries
import numpy as np
import numpy.random as nprand
import math as math
import pandas as pd 

In [2]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
#!python3 -m spacy download en
#!pip install -U gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sahri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#!pip install gensim
#!pip install spacy

In [4]:
#!pip install pyLDAvis

In [5]:
import re
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
#Using Stop Words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
#read iris.csv
import pandas as pd
df = pd.read_csv('security.csv',encoding='utf-8')
df.head()

,postid,tags,title,question,answers,words,creation_date,Related
0,40471439,blockchain diffie-hellman encryption javascript,js diffi hellman ssl blockchain auth,tri figur secur qualiti ssl mimick without nee...,NaN,encrypt|scripting|diffie hellman|ssl|security|...,2016-11-07 17:52:22.793 UTC,Yes
1,32771909,cryptography jar jarsigner java smartcard,sign jar smartcard,use pkcs11 smartcard work jarsign sign jar fil...,ad -sigalg algortihm avail [sha256withrsa case...,crypt|signing|certificate|validate|private key...,2015-09-24 22:23:45.947 UTC,Yes
2,15574818,java jespa ntlm servlets spring-security,cannot creat session respons commit ie7 ie8,work applic allow user login use ntlm form log...,NaN,security|login|credential|password|validate|au...,2013-03-22 16:04:18.12 UTC,Yes
3,19042957,asp.net c# security,strong name assembl,thought learn strong name creat one exampl cac...,refer strong-nam assembl normal assembl includ...,security|signing|public key|signature|private key,2013-09-27 04:51:44.143 UTC,Yes
4,59694966,ssh digital-ocean cyberduck,eof read packet,connect via sftp digitalocean vps username/pas...,need updat password termin window *open digit ...,sftp|username|password|authentication|login,11/1/2020 13:43,Yes


In [8]:
# Convert to list
data = df.words.values.tolist()

# Remove Emails
#data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
#data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
#data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:4])

['encrypt|scripting|diffie '
 'hellman|ssl|security|authentication|certificate|authorise|public key',
 'crypt|signing|certificate|validate|private key|dsa|rsa|availability',
 'security|login|credential|password|validate|authentication',
 'security|signing|public key|signature|private key']


In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(len(data_words))


#print(data_words[:10])

385


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[100]]])

['security', 'openid', 'oauth', 'login', 'authorise', 'authentication', 'validate', 'hack']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
#!python -m spacy download en_core_web_sm

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm",disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:10])

[['encrypt', 'security', 'authentication', 'certificate', 'authorise', 'public', 'key'], ['sign', 'certificate', 'validate', 'private', 'dsa', 'rsa', 'availability'], ['password', 'validate', 'authentication'], ['security', 'sign', 'public', 'key', 'signature', 'private', 'key'], ['password', 'authentication', 'login'], ['validate'], ['delegation', 'transport', 'security', 'privilege', 'authentication'], ['security', 'encrypt'], ['security', 'transport', 'certificate', 'trust', 'validate', 'insecure', 'bypass'], ['certificate', 'signature', 'hash', 'encode', 'validate']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [15]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('authentication', 1),
  ('authorise', 1),
  ('certificate', 1),
  ('encrypt', 1),
  ('key', 1),
  ('public', 1),
  ('security', 1)]]

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.278*"script" + 0.157*"forge" + 0.132*"inject" + 0.081*"xss" + '
  '0.051*"checksum" + 0.039*"md" + 0.039*"account" + 0.036*"integrity" + '
  '0.033*"hash" + 0.015*"crc"'),
 (1,
  '0.191*"encrypt" + 0.151*"key" + 0.077*"hash" + 0.074*"public" + '
  '0.072*"private" + 0.063*"signature" + 0.045*"salt" + 0.043*"signing" + '
  '0.037*"encode" + 0.031*"scripting"'),
 (2,
  '0.226*"security" + 0.119*"oauth" + 0.103*"scripting" + 0.095*"validate" + '
  '0.085*"signing" + 0.054*"safe" + 0.043*"danger" + 0.042*"protect" + '
  '0.033*"unsafe" + 0.027*"sanitise"'),
 (3,
  '0.321*"authorise" + 0.207*"authentication" + 0.191*"security" + '
  '0.093*"crypt" + 0.063*"encode" + 0.015*"control" + 0.014*"decode" + '
  '0.009*"day" + 0.001*"password" + 0.001*"username"'),
 (4,
  '0.183*"trust" + 0.119*"integrity" + 0.097*"sign" + 0.093*"hack" + '
  '0.075*"tls" + 0.049*"hmac" + 0.046*"csrf" + 0.042*"safe" + '
  '0.039*"authentication" + 0.026*"audit"'),
 (5,
  '0.282*"attack" + 0.114*"security" 

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -4.420951309771526

Coherence Score:  0.4389590941384503


In [19]:
lda_model[corpus[0]]

([(1, 0.53513676),
  (2, 0.017106641),
  (3, 0.34779343),
  (7, 0.041633077),
  (9, 0.015095048)],
 [(0, [3, 7]), (1, [3]), (2, [1]), (3, [1]), (4, [1]), (5, [1]), (6, [3, 1])],
 [(0, [(3, 0.9852767), (7, 0.014720595)]),
  (1, [(3, 0.9999983)]),
  (2, [(1, 0.9994411)]),
  (3, [(1, 0.9979032)]),
  (4, [(1, 0.9999975)]),
  (5, [(1, 0.9999949)]),
  (6, [(1, 0.20224243), (3, 0.79125863)])])

In [20]:
#### Visualize the topics
pyLDAvis.enable_notebook()
vis =gensimvis.prepare(lda_model, corpus, id2word)
vis

C:\Users\sahri\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.254399  0.093867       1        1  25.637399
1      0.001863  0.323094       2        1  19.389312
2     -0.073579  0.020897       3        1  10.294167
9      0.280477  0.128761       4        1   8.329800
3     -0.208420 -0.041838       5        1   7.189835
6     -0.094220 -0.140553       6        1   6.582489
8      0.017569 -0.175901       7        1   6.499785
5     -0.038758 -0.034926       8        1   6.379807
4      0.173984 -0.110317       9        1   5.293025
0      0.195483 -0.063084      10        1   4.404381, topic_info=              Term        Freq       Total Category  logprob  loglift
13        password   88.000000   88.000000  Default  30.0000  30.0000
0   authentication  101.000000  101.000000  Default  29.0000  29.0000
11            sign   52.000000   52.000000  Default  28.0000  28.0000
1        authorise   44.000000   44.000000  Default  27.0000  27.0000
12        validate   55.000000   55.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
15           login    0.092837   78.369116  Topic10  -6.6725  -3.6158
65          tamper    0.092836    6.270663  Topic10  -6.6725  -1.0902
34         protect    0.092834   30.865297  Topic10  -6.6726  -2.6840
24            safe    0.092833   32.740116  Topic10  -6.6726  -2.7430
62           pbkdf    0.092833    7.813534  Topic10  -6.6726  -1.3102

[463 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
54        6  0.622495         access
40       10  0.814645        account
114       8  0.504294    adversarial
63        2  0.926369            aes
78        8  0.669962          asset
...     ...       ...            ...
101       8  0.886802        violate
72        2  0.656512          virus
30        3  0.252171  vulnerability
30        8  0.672456  vulnerability
94       10  0.880950            xss

[165 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 3, 10, 4, 7, 9, 6, 5, 1])